Primeiro _prompt_ enviado ao ChatGpt:


Gere em python um algoritmo que observe os seguintes requisitos:
1. crie um algoritmo de árvore de decisão from scratch;
2. não utilize bibliotecas como scikit learn ou quaisquer outras que implementem tal algoritmo;
3. utilize a heurística top-down;
4. utilize uma estratégia recursiva e gulosa (greedy);
5. utilize quando necessário bibliotecas de manipulaçao de dados como Pandas e Numpy;
6. os atributos poderão ser tanto categóricos, como numéricos (inteiro e float);


Resposta do site:

Aqui está uma implementação de um algoritmo de árvore de decisão from scratch que atende os requisitos:

Top-down: O algoritmo começa da raiz e segue até as folhas.
Recursivo e guloso (greedy): Ele escolhe, em cada nó, a melhor divisão (feature) com base na redução de entropia ou ganho de informação, e faz isso recursivamente para cada subdivisão.
A implementação usa Numpy para manipulação de dados e cálculos de ganho de informação, e Pandas para manipulação de datasets.

In [1]:
import numpy as np
import pandas as pd

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X:pd.DataFrame, y):
        dataset = X.copy()
        dataset['label'] = y
        self.tree = self._build_tree(dataset)

    def predict(self, X:pd.DataFrame):
        return X.apply(self._predict_row, axis=1)

    def _entropy(self, y):
        # Calcula a entropia dos rótulos
        unique_classes, class_counts = np.unique(y, return_counts=True)
        probabilities = class_counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities))

    def _information_gain(self, left_y, right_y, parent_entropy):
        # Calcula o ganho de informação após uma divisão
        left_weight = len(left_y) / (len(left_y) + len(right_y))
        right_weight = len(right_y) / (len(left_y) + len(right_y))
        child_entropy = (left_weight * self._entropy(left_y) +
                         right_weight * self._entropy(right_y))
        return parent_entropy - child_entropy

    def _best_split(self, dataset):
        # Encontra a melhor divisão
        best_gain = 0
        best_feature = None
        best_split_value = None
        parent_entropy = self._entropy(dataset['label'])
        
        for feature in dataset.columns[:-1]:
            values = dataset[feature].unique()
            for value in values:
                left_split = dataset[dataset[feature] <= value]
                right_split = dataset[dataset[feature] > value]
                
                if len(left_split) == 0 or len(right_split) == 0:
                    continue
                
                gain = self._information_gain(left_split['label'], right_split['label'], parent_entropy)
                
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_split_value = value
        
        return best_feature, best_split_value, best_gain

    def _build_tree(self, dataset, depth=0):
        # Função recursiva para construir a árvore de decisão
        X = dataset.drop(columns=['label'])
        y = dataset['label']
        
        # Critérios de parada
        if len(np.unique(y)) == 1:
            return np.unique(y)[0]  # Folha com uma única classe
        if len(X) < self.min_samples_split or (self.max_depth is not None and depth >= self.max_depth):
            return y.mode()[0]  # Folha com a classe majoritária
        
        # Melhor divisão
        best_feature, best_split_value, best_gain = self._best_split(dataset)
        
        if best_gain == 0:
            return y.mode()[0]  # Folha com a classe majoritária
        
        left_split = dataset[dataset[best_feature] <= best_split_value]
        right_split = dataset[dataset[best_feature] > best_split_value]
        
        # Nós filhos
        node = {
            'feature': best_feature,
            'value': best_split_value,
            'left': self._build_tree(left_split, depth + 1),
            'right': self._build_tree(right_split, depth + 1)
        }
        
        return node

    def _predict_row(self, row):
        node = self.tree
        while isinstance(node, dict):
            if row[node['feature']] <= node['value']:
                node = node['left']
            else:
                node = node['right']
        return node


### **Detalhes do Algoritmo:**
- Heurística Top-Down: O algoritmo começa dividindo o dataset a partir da raiz (nó inicial), calculando a entropia de cada feature para determinar qual delas traz maior ganho de informação e faz a divisão baseada nesse cálculo. Esse processo é repetido recursivamente.

- Estratégia Recursiva e Gulosa: A divisão de cada nó é escolhida de forma gulosa, isto é, para cada nó, o algoritmo tenta encontrar a melhor divisão naquele momento, sem considerar o impacto da escolha no futuro.

### **Parâmetros:**
`max_depth`: Limita a profundidade da árvore para evitar overfitting.
`min_samples_split`: Define o número mínimo de amostras para permitir a divisão do nó.

### **Exemplo de uso:**

In [2]:
# Exemplo de uso com dataset fictício
data = {'feature1': [1, 2, 2, 3, 3],
        'feature2': [1, 1, 2, 2, 3],
        'label': [0, 0, 1, 1, 1]}
df = pd.DataFrame(data)
X = df.drop(columns=['label'])
y = df['label']

tree = DecisionTree(max_depth=3)
tree.fit(X, y)

# Predições
predictions = tree.predict(X)
print(predictions)

0    0
1    0
2    1
3    1
4    1
dtype: int64
